# Unit 1: Set Indoor Navigation Stack

The first thing you will need for a robot that patroles is the ability to move around without crashing into everything. For this you will learn:<br>
* How to move the robot
* How to set the navigation stack to be able to send him a location and be able to reach that destination.
* How to send it a sequence of waypoints and execute those movements.

## Move Summit XL

In this case you have the standard wheel model with skid-steering. In escence this make it a normal vehicle that has manouver much more to get to a location and orientation. In th eproject you will use the omnidirectional wheels model.

To move it you just have to publish in the topic **/summit_xl_control/cmd_vel**.

<p style="background:#EE9023;color:white;">Exercise U1-1</p>

Publish through the web shell in the mentioned topic and see if you can make SummitXL move forwards, backwards, left, right and in circles.

<p style="background:#EE9023;color:white;">END Exercise U1-1</p>

You should have done something like this:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1 to move Haro</p><br>
rostopic pub /summit_xl_control/cmd_vel geometry_msgs/Twist [TAB][TAB]
</th>
</tr>
</table>

With this command you can then fill in the structure that appears after hitting TAB key two consecutive times, to publish the message.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1 to move Haro</p><br>
rostopic info /summit_xl_control/cmd_vel
</th>
</tr>
</table>

Like this you would get the type of message to publish in that topic.

As you can see the type of the message is the classical **geometry_msgs/Twist**. It will move as any differential drive that you have used. That is you move it by only publishing in the **linear.x** and the **angular.z**.<br>
Of ocurse this is not the way you will use it. You normally need to move it manually to make tests, so the best way is to create a keyboard teleop system that can move the SummitXL through the keyboard or any other similar device.<br>
Here you have an example of launch file that could be used for this matter:

In [ ]:
<launch>
  <!-- turtlebot_teleop_key already has its own built in velocity smoother -->
  <node pkg="turtlebot_teleop" type="turtlebot_teleop_key.py" name="turtlebot_teleop_keyboard"  output="screen">
    <param name="scale_linear" value="0.5" type="double"/>
    <param name="scale_angular" value="1.5" type="double"/>
    <remap from="turtlebot_teleop_keyboard/cmd_vel" to="/summit_xl_control/cmd_vel"/>
  </node>
</launch>

There two point to talk about here:<br>
* turtlebot_teleop_key.py: This is the program used to capture the keys pressed in the terminal and converts them to Twist commands that are then published into the topic **turtlebot_teleop_keyboard/cmd_vel**

* remap: This remap is to transfer all the information published in **turtlebot_teleop_keyboard/cmd_vel** in the topic that SummitXL controller is listening to **/summit_xl_control/cmd_vel**.

Of ocurse you could publish directly into **/summit_xl_control/cmd_vel**, but then you would have to create your own version of **turtlebot_teleop_key.py**. Thanks to the ROS infraestructure, this way you recicle code, and get work done faster.

<p style="background:#EE9023;color:white;">Exercise U1-2</p>

Create a package that you  will use along the rest of this course where you can have all your programs. Use the name **my_sumit_xl_tools** for future refference.<br>
Then create your own launch for keyboard teleoperation based on the example and play around with SummitXL. See how it moves and its limitations. This is crucial to know what you can do with your robot.

<p style="background:#EE9023;color:white;">END Exercise U1-2</p>

## Navigate the Summit Robot!

To finish this project successfully, we provide below the 5 steps you should follow with clear instructions and even solutions.

## What does Summit provide for programming it?

#### Sensors

* <b>Laser sensor</b>: Summit has a <a href="https://en.wikipedia.org/wiki/Inertial_measurement_unit">Hokuyo Laser</a> which provides information about the objects that detects in it's range. Value of the sensor is provided through the topic <span class="ign_green"><i>/hokuyo_base/scan</i></span>
* <b>Odometry</b>: the odometry of the robot can be accessed through the <span class="ign_green"><i>/odom</i></span> topic

#### Actuators

* <b>Speed</b>: you can send speed commands to move the robot through the topic <span class="ign_green"><i>/summit_xl_control/cmd_vel</i></span>.

## Steps you should cover

1. <a href="#step1">Step 1: Generate a Map of the environment (Dedicate 2 hours)</a>
2. <a href="#step2">Step 2: Make the Robot Localize itlsef in the Map you've created (Dedicate 2 hours)</a>
3. <a href="#step3">Step 3: Set Up a the Path Planning System (Dedicate 3 hours)</a>
4. <a href="#stepExtra">Step 4: Create a ROS program that interacts with the Navigation Stack (Dedicate 3 hours)</a>

### Step 1: Generate a Map of the environment

1. Make sure the Summit robot is publishing it's transform data correctly.
2. Create a package called <span class="ign_green"><i>my_summit_mapping</i></span> that will contain all the files related to Mapping.
3. Create a <span class="ign_green">launch file</span> that will launch the slam_gmapping node and add the necessary parameters in order to properly configure the slam_gmapping node.
4. Launch the slam_gmapping node and create a map of the simulated environment.
5. Save the map you've just created.
6. Create a script that automatically saves maps.
7. Create a <span class="ign_green">launch file</span> that will provide the created map to other nodes.

So, let's start doing them!

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">1. Generate the necessary files in order to visualize the frames tree.</p>
</th>
</tr>
</table>

You should get a file like this one:

<img src="img/summit_xl_instruction_manual_unit1_summit_frame_tree.png" width="900"></img>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create a package called <i>my_summit_mapping.</i></p>
</th>
</tr>
</table>

Create the package adding <b>rospy</b> as only dependency.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the launch file for the gmapping node.</p>
</th>
</tr>
</table>

In in this step, you'll have to create a launch file for the slam_gmapping node and add the parameters you think you need to set.

Here you can see a full list of parameters you can set to configure the slam_gmapping node: http://docs.ros.org/hydro/api/gmapping/html/

Here you have an example launch file for the gmapping node:

In [ ]:
<launch>
    <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping" output="screen">
      <!-- simulation remap from="scan" to="/hokuyo_laser_topic"/ -->
      <!-- real -->
      <!-- remap from="scan" to="/scan_filtered"/ -->
      <remap from="scan" to ="/hokuyo_base/scan"/>
      <!-- param name="map_udpate_interval" value="5.0"/ -->
      <param name="base_frame" value="base_footprint"/>
      <param name="odom_frame" value="odom"/>
      <param name="map_udpate_interval" value="5.0"/>
      <param name="maxUrange" value="2.0"/>
      <param name="sigma" value="0.05"/>
      <param name="kernelSize" value="1"/>
      <param name="lstep" value="0.05"/>
      <param name="astep" value="0.05"/>
      <param name="iterations" value="5"/>
      <param name="lsigma" value="0.075"/>
      <param name="ogain" value="3.0"/>
      <param name="lskip" value="0"/>
      <param name="srr" value="0.1"/>
      <param name="srt" value="0.2"/>
      <param name="str" value="0.1"/>
      <param name="stt" value="0.2"/>
      <param name="linearUpdate" value="0.2"/>
      <param name="angularUpdate" value="0.1"/>
      <param name="temporalUpdate" value="3.0"/>
      <param name="resampleThreshold" value="0.5"/>
      <param name="particles" value="100"/>
      <param name="xmin" value="-50.0"/>
      <param name="ymin" value="-50.0"/>
      <param name="xmax" value="50.0"/>
      <param name="ymax" value="50.0"/>
      <param name="delta" value="0.05"/>
      <param name="llsamplerange" value="0.01"/>
      <param name="llsamplestep" value="0.01"/>
      <param name="lasamplerange" value="0.005"/>
      <param name="lasamplestep" value="0.005"/>
    </node>
</launch>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">4. Launch the node using the launch file you've just created, and create a map of the environment.</p>
</th>
</tr>
</table>

In order to move the robot around the environment, you can use the keyboard teleop. To launch the keyboard teleop, just execute the following command:

Also remember to launch Rviz and add the proper displays in order to visualize the map you are generating. You should get in RViz something like this:

<img src="img/max_urange_2.png" width="800"></img>

Now start moving the robot around the room to create the map.

What's happening? Are you finding difficulties in order to generate a proper Map? Is there anything strange in the Mapping process? Do you know what could be the reason of it?

What's happening here is the following:

1. Your maxUrange parameter in the launch file of the slam_gmapping node is set to 2. This parameter sets the maximum range of the laser scanner that is used for map building. Since this is a small value, the robot can't get enough data to know where it is, so it may get lost. Tis will cause some strange issues during the mapping process, such as the robot readjusting its position in a wrong way.
<br><br>
2. In order to solve it, you should set the maxUrange parameter to some higher value, for instance, 7. This way, the robot will be able to get more data of the environemnt, so it won't get lost. Now, you'll be able to finalize the mapping process correctly.

Once you modify this parameter and relaunch the slam_gmapping node, you should get something like this in RViz:

<img src="img/max_urange_7.png" width="800"></img>

Now, you will be able to map the environment much more easily.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">5. Save the map.</p>
</th>
</tr>
</table>

Create a directory in your package named <b>maps</b>, and save the map files here. Your map image file should look something similar to this:

<img src="img/fetch_map.png" width="600"></img>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">6. Create a script that automatically saves different maps.</p>
</th>
</tr>
</table>

Inside your package, create a directory called <b>src</b>. Inside this directory, create an script that will do the following:

1. It will launch a node named <b>map_recorder</b>
2. The node will create a Subscriber that will start reading to the <b>/odom</b> topic, and will check the distance moved by the robot.
3. Each time the robot moves 5 meters, the script will call the <b>map_saver</b> in order to save the generated map until that moment.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">7. Create a lanch file that launches the map_server node.</p>
</th>
</tr>
</table>

As you've also seen in the course, the map you've just created will be used by other nodes (this means, it'll be used in further steps) in order to perform navigation. 
<br><br>
Therefore, you'll need to create a launch file in order to provide the map. As you know, this is done through the map_server node.

Next you have an example of how this launch fille should look like.

In [ ]:
<launch>
    <node name="map_server" pkg="map_server" type="map_server" args="$(find my_summit_mapping)/maps/my_map.yaml"/>
</launch>

Finally, launch this file and check that it's really providing the map.

### Step 2: Make the Robot Localize itself in the Map

This step has 4 actions for you to do:<br>

1. Create a package called <span class="ign_green"><i>my_summit_localization</i></span> that will contain all the files related to Localization.
2. Create a <span class="ign_green">launch file</span> that will launch the amcl node and add the necessary parameters in order to properly configure the amcl node.
3. Launch the node and check that the robot properly localizes itself in the environment.
4. Create a table with three different spots.
5. Create a ROS Service to save the different spots into a file.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">1. Create a package called my_summit_localization.</p>
</th>
</tr>
</table>

Create the package adding <b>rospy</b> as only dependency.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the launch file for the amcl node.</p>
</th>
</tr>
</table>

In the Localization section (Chapater 3) of this course you've seen how to create a launch file for the amcl node. You've also seen some of the most important parameters to set. So in in this step, you'll have to create a launch file for the amcl node and add the parameters you think you need to set.

Here you can see a full list of parameters you can set to configure the amcl node: http://wiki.ros.org/amcl

Remember that, before setting the amcl node parameters, you'll need to load the map created in Step 1. For that, you'll just need to include in the amcl node launch file, the launch you've created in Step 1 in order to provide the map to other nodes.

Here you have an example launch file for the gmapping node:

In [ ]:
<launch>

<!-- Run the map server -->
<include file="$(find my_summit_gma)/launch/map_server.launch" />

<node pkg="amcl" type="amcl" name="amcl" output="screen">

  <remap from="scan" to="/hokuyo_base/scan" />
  <remap from="cmd_vel" to="/summit_xl_controller/cmd_vel"/>

  <!-- Publish scans from best pose at a max of 10 Hz -->
  <param name="odom_model_type" value="diff"/>
  <param name="odom_alpha5" value="0.1"/>
  <param name="transform_tolerance" value="0.2" />
  <param name="gui_publish_rate" value="10.0"/>
  <param name="laser_max_beams" value="30"/>
  <param name="min_particles" value="500"/>
  <param name="max_particles" value="5000"/>
  <param name="kld_err" value="0.05"/>
  <param name="kld_z" value="0.99"/>
  <param name="odom_alpha1" value="0.2"/>
  <param name="odom_alpha2" value="0.2"/>
  <param name="odom_alpha3" value="0.8"/>
  <param name="odom_alpha4" value="0.2"/>
  <param name="laser_z_hit" value="0.5"/>
  <param name="laser_z_short" value="0.05"/>
  <param name="laser_z_max" value="0.05"/>
  <param name="laser_z_rand" value="0.5"/>
  <param name="laser_sigma_hit" value="0.2"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_model_type" value="likelihood_field"/>
  <param name="laser_likelihood_max_dist" value="2.0"/>
  <param name="update_min_d" value="0.2"/>
  <param name="update_min_a" value="0.5"/>
  <param name="odom_frame_id" value="odom"/>
  <param name="base_frame_id" value="base_footprint"/>
  <param name="global_frame_id" value="map"/>
  <param name="resample_interval" value="1"/>
  <param name="transform_tolerance" value="0.1"/>
  <param name="recovery_alpha_slow" value="0.0"/>
  <param name="recovery_alpha_fast" value="0.0"/>
  <param name="initial_pose_x" value ="0.0"/>
  <param name="initial_pose_y" value ="0.0"/>
  <param name="initial_pose_a" value ="0.0"/>

</node>

</launch>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Launch the node and check that the Summit robot correctly localizes itself in the map.</p>
</th>
</tr>
</table>

Launch the node and check in RViz that the Summit robot correctly localizes itseld in the Map.

If you've done all the previous steps correctly, you should see in Rviz something like this:

<img src="img/summit_localization.png" width="600"></img>

In order to check that the localization works fine, move the robot around the environment and check that the particle cloud keeps getting smaller the more you move the robot.

<img src="img/summit_localization2.png" width="600"></img>

<img src="img/summit_localization3.png" width="600"></img>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">4. Create the spots table.</p>
</th>
</tr>
</table>

Once you've checked that localization is working fine, you'll need to create a table with 3 different spots in the environemnt. For each point, assign a tag (with the name of the spot) alongside its coordinates in the Map.

The 3 spots you'll have to register into the table are the next ones:

* Fetch Room Center
<br>
<img src="img/fetch_room.png" width="500"></img>
<br>
* Facing the Turtle's Poster
<br>
<img src="img/turtle_poster.png" width="500"></img>
* Besides the Table
<br>
<img src="img/besides_table.png" width="500"></img>
<br>

You can access the coordinates of each position by looking into the topics the amcl node publishes on. The only data you really need to save are the position and orientation. 

Next you have a screenshot of the <i>/amcl_pose</i> topic:

<img src="img/echo_amcl_pose.png" width="800"></img>

Create a file named <b>spots.txt</b> and write in the pose data you've get from the 3 spots.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">5. Create a ROS Service that saves the different spots into a file.</p>
</th>
</tr>
</table>

In the previous section, you created a file with the coordinates of the 3 spots. But the whole process was too cumbersome, don't you think so? Why don't we create a ROS Service that does the work for us? Sounds like a good idea, right? Then let's do it!

So now, you'll create a ROS program that will do the following:

1. It will launch a node named <b>spot_recorder</b>.
2. This node will contain a Service Server named <b>/save_spot</b>, that will take an string as input.
3. When this service is called, it will store the current coordinates of the robot (position and orientation values) alongside with a label, that will be the string provided on the service call.
4. When the <i>end</i> string is provided in the service call, the node will write all the values stored into a file named <b>spots.txt</b>.
5. Finally, the service will return a message indicating if the file has been saved correctly.

In order to achieve this, let's divide it into smaller parts.

#### 1 Service Message

First of all, you'll have to determine the kind of data you need for your service message.

1. Determine what input data you need ( <span class="ign_blue"><b>request</b></span> )
2. Determine what data you want the service to return ( <span class="ign_blue"><b>response</b></span> )

Next, search if there exists an already built message in the system that suits your needs. If it doesn't, then you'll have to create your own custom message with the data you want.

If this is the case, then do the following. Inside the pacakge you just created, create a new directory named <b>srv</b>. Inside this directory, create a file named <b>SumitXLPose.srv</b>, that will contain the definition of your service message. 

This file could be something like this:

Now you'll have to modify the <b>package.xml</b> and <b>CMakeLists.txt</b> files of your package, in order to compile the new message. Assuming the only dependency you added to your package is <b>rospy</b>, you should do the following modifications to these files:

#### In CMakeLists.txt

You will have to edit/uncomment four functions inside this file:

* find_package()
* add_service_files()
* generate_messages()
* catkin_package()

add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

generate_messages(
  DEPENDENCIES
  std_msgs
)

#### In package.xml:

Add this 2 lines to the file:

When you've finshed with the modifications, compile your package and check that your message has been correctly compiled and is ready to use. In order to check this, you can use the command <b><i>rossrv show MyServiceMessage</i></b>.

catkin_make output:

<img src="img/catkin_make_output.png" width="600"></img>

rossrv show output:

<img src="img/service_msg_output.png" width="600"></img>

#### 2 Service Code

Once your message is created, you're ready to use it in your service! So, let's write the code for our service. Inside the src directory of your package, create a file named <b>spots_to_file.py</b>. Inside this file, write the code necessary for you Service.

In [ ]:
#! /usr/bin/env python
import os
import rospy
from sumit_xl_tools.srv import SumitXLPose, SumitXLPoseResponse, SumitXLPoseRequest
from geometry_msgs.msg import PoseWithCovarianceStamped
import rospkg

"""
# topic: /amcl_pose
# geometry_msgs/PoseWithCovarianceStamped 

std_msgs/Header header                                                                                                                                       
  uint32 seq                                                                                                                                                 
  time stamp                                                                                                                                                 
  string frame_id                                                                                                                                            
geometry_msgs/PoseWithCovariance pose                                                                                                                        
  geometry_msgs/Pose pose                                                                                                                                    
    geometry_msgs/Point position                                                                                                                             
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
    geometry_msgs/Quaternion orientation                                                                                                                     
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
      float64 w                                                                                                                                              
  float64[36] covariance
"""

class SummitXLPose(object):
    def __init__(self):
        self.pose_service = rospy.Service('/get_pose_sumitxl_service', SumitXLPose , self.pose_service_callback) # create the Service called my_service with the defined callback
        rospy.Subscriber("/amcl_pose", PoseWithCovarianceStamped, self.amcl_pose_callback)
        self.pose_now = PoseWithCovarianceStamped()
        
        rospack = rospkg.RosPack()
        self.spot_file_path = rospack.get_path('my_summit_localization') + "/spots/spots_saved.txt"
        
        try:
            os.remove(self.spot_file_path)
        except:
            rospy.loginfo("File Not Found "+str(self.spot_file_path) )
        # Init File for Spot saving
        file2write=open(self.spot_file_path,'a')
        file2write.write("### This is the Spot Saving File ###\n")
        file2write.close()
        
    def start_loop_service(self):
        rospy.spin() # mantain the service open.
        
    def pose_service_callback(self, request):
        spot_name = request.label
        
        with open(self.spot_file_path, "a") as myfile:
            spot_pose = self.pose_now.pose.pose
            myfile.write(spot_name+":\n"+str(spot_pose)+"\n\n")

        response = SumitXLPoseResponse()
        response.navigation_successfull = True
        response.message = "Spot <"+spot_name+"> Saved"

        return response
    
    def amcl_pose_callback(self, message):
        self.pose_now = message
        


def startPoseService():
    rospy.init_node('sumitxl_poseservice_server')
    sumitpose = SummitXLPose()
    sumitpose.start_loop_service()

if __name__ == "__main__":
    startPoseService()
    

#### 3 Launch file

Create a launch file for the node you've just created. You can also launch this node in the same launch file you've created to launch the slam_gmapping node. It's up to you.

#### 4 Test it

Using the keyboard teleop, move the robot to the 3 different spots showed above. At each one of these spots, perform a service call to the service you've just created. In the service call, provide the string with the name you want to give to each spot.
<br><br>
For example, when you are at the center of the Fetch Room, call your service like this:

Repeat this for the 3 different spots. When finished, do a last service call providing <i>end</i> as the string in order to create the file.

### Step 3: Set Up the Path Planning System

This step has 5 actions for you to do:

1. Create a package called <span class="ign_green"><i>my_summit_path_planning</i></span> that will contain all the files related to Path Planning.
2. Create a <span class="ign_green">launch file</span> that will launch the move_base node.
3. Create the necessary parameter files in order to properly configure the move_base node.
4. Create the necessary parameter files in order to properly configure the global and local costmaps.
5. Create the necessary parameter files in order to properly configure the global and local planners.
6. Launch the node and check everything works fine.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">1. Create a package named my_summit_path_planning.</p>
</th>
</tr>
</table>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the launch file for the move_base node.</p>
</th>
</tr>
</table>

In the Path Planning section (Chapater 4) of this course you've seen how to create a launch file for the move_base node. So in in this step, you'll have to create a launch file for the move_base node and add the parameters and the parameter files required.

Here you have an example of the move_base.launch file:

In [ ]:
<?xml version="1.0"?>
<!-- NEW SUMIT XL NAVIGATION -->
<launch>
        
    <include file="$(find my_summit_localization)/launch/amcl.launch" />
    
    <remap from="cmd_vel" to="/move_base/cmd_vel" />
    
    <arg name="no_static_map" default="false"/>
    
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->
    
    <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
        
        <param name="base_global_planner" value="$(arg base_global_planner)"/>
        <param name="base_local_planner" value="$(arg base_local_planner)"/>  
        <rosparam file="$(find sumit_xl_tools)/new_config/planner.yaml" command="load"/>
        
        <!-- observation sources located in costmap_common.yaml -->
        <rosparam file="$(find my_summit_path_planning)/new_config/costmap_common.yaml" command="load" ns="global_costmap" />
        <rosparam file="$(find my_summit_path_planning)/new_config/costmap_common.yaml" command="load" ns="local_costmap" />
        
        <!-- local costmap, needs size -->
        <rosparam file="$(find my_summit_path_planning)/new_config/costmap_local.yaml" command="load" ns="local_costmap" />
        <param name="local_costmap/width" value="5.0"/>
        <param name="local_costmap/height" value="5.0"/>
        
        <!-- static global costmap, static map provides size -->
        <rosparam file="$(find my_summit_path_planning)/new_config/costmap_global_static.yaml" command="load" ns="global_costmap" unless="$(arg no_static_map)"/>
        
        <!-- global costmap with laser, for odom_navigation_demo -->
        <rosparam file="$(find my_summit_path_planning)/new_config/costmap_global_laser.yaml" command="load" ns="global_costmap" if="$(arg no_static_map)"/>
        <param name="global_costmap/width" value="100.0" if="$(arg no_static_map)"/>
        <param name="global_costmap/height" value="100.0" if="$(arg no_static_map)"/>
    </node>

</launch>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the move_base .yaml files</p>
</th>
</tr>
</table>

Here you have an example of the **planner.yaml** file:

In [ ]:
controller_frequency: 5.0
recovery_behaviour_enabled: true

NavfnROS:
  allow_unknown: true # Specifies whether or not to allow navfn to create plans that traverse unknown space.
  default_tolerance: 0.1 # A tolerance on the goal point for the planner.

TrajectoryPlannerROS:
  # Robot Configuration Parameters
  acc_lim_x: 2.5
  acc_lim_theta:  3.2

  max_vel_x: 1.0
  min_vel_x: 0.0

  max_vel_theta: 1.0
  min_vel_theta: -1.0
  min_in_place_vel_theta: 0.2

  holonomic_robot: false
  escape_vel: -0.1

  # Goal Tolerance Parameters
  yaw_goal_tolerance: 0.1
  xy_goal_tolerance: 0.2
  latch_xy_goal_tolerance: false

  # Forward Simulation Parameters
  sim_time: 2.0
  sim_granularity: 0.02
  angular_sim_granularity: 0.02
  vx_samples: 6
  vtheta_samples: 20
  controller_frequency: 20.0

  # Trajectory scoring parameters
  meter_scoring: true # Whether the gdist_scale and pdist_scale parameters should assume that goal_distance and path_distance are expressed in units of meters or cells. Cells are assumed by default (false).
  occdist_scale:  0.1 #The weighting for how much the controller should attempt to avoid obstacles. default 0.01
  pdist_scale: 0.75  #     The weighting for how much the controller should stay close to the path it was given . default 0.6
  gdist_scale: 1.0 #     The weighting for how much the controller should attempt to reach its local goal, also controls speed  default 0.8

  heading_lookahead: 0.325  #How far to look ahead in meters when scoring different in-place-rotation trajectories
  heading_scoring: false  #Whether to score based on the robot's heading to the path or its distance from the path. default false
  heading_scoring_timestep: 0.8   #How far to look ahead in time in seconds along the simulated trajectory when using heading scoring (double, default: 0.8)
  dwa: true #Whether to use the Dynamic Window Approach (DWA)_ or whether to use Trajectory Rollout
  simple_attractor: false
  publish_cost_grid_pc: true  

  # Oscillation Prevention Parameters
  oscillation_reset_dist: 0.25 #How far the robot must travel in meters before oscillation flags are reset (double, default: 0.05)
  escape_reset_dist: 0.1
  escape_reset_theta: 0.1

DWAPlannerROS:
  # Robot configuration parameters  
  acc_lim_x: 2.5
  acc_lim_y: 0
  acc_lim_th: 3.2

  max_vel_x: 0.5
  min_vel_x: 0.0
  max_vel_y: 0
  min_vel_y: 0

  max_trans_vel: 0.5
  min_trans_vel: 0.1
  max_rot_vel: 1.0
  min_rot_vel: 0.2

  # Goal Tolerance Parameters
  yaw_goal_tolerance: 0.1
  xy_goal_tolerance: 0.2
  latch_xy_goal_tolerance: false

  # # Forward Simulation Parameters
  # sim_time: 2.0
  # sim_granularity: 0.02
  # vx_samples: 6
  # vy_samples: 0
  # vtheta_samples: 20
  # penalize_negative_x: true

  # # Trajectory scoring parameters
  # path_distance_bias: 32.0 # The weighting for how much the controller should stay close to the path it was given
  # goal_distance_bias: 24.0 # The weighting for how much the controller should attempt to reach its local goal, also controls speed
  # occdist_scale: 0.01 # The weighting for how much the controller should attempt to avoid obstacles
  # forward_point_distance: 0.325 # The distance from the center point of the robot to place an additional scoring point, in meters
  # stop_time_buffer: 0.2  # The amount of time that the robot must stThe absolute value of the veolicty at which to start scaling the robot's footprint, in m/sop before a collision in order for a trajectory to be considered valid in seconds
  # scaling_speed: 0.25 # The absolute value of the veolicty at which to start scaling the robot's footprint, in m/s
  # max_scaling_factor: 0.2 # The maximum factor to scale the robot's footprint by

  # # Oscillation Prevention Parameters
  # oscillation_reset_dist: 0.25 #How far the robot must travel in meters before oscillation flags are reset (double, default: 0.05)

If you are not sure of what all these values are, please go over **RobotIgniteAcademy** Navigation Course in ROS.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">4. Create the costmap_common_params.yaml, global_costmap_params.yaml and local_costmap_params.yaml files</p>
</th>
</tr>
</table>

Here you have an example of the **costmap_common.yaml** file:

In [ ]:
footprint: [[0.35, -0.3], [0.35, 0.3], [-0.35,0.3], [-0.35, -0.3]]
footprint_padding: 0.01

robot_base_frame: base_link
update_frequency: 4.0
publish_frequency: 3.0
transform_tolerance: 0.5

resolution: 0.05

obstacle_range: 5.5
raytrace_range: 6.0

#layer definitions
static:
    map_topic: /map
    subscribe_to_updates: true

obstacles_laser:
    observation_sources: hokuyo_laser
    hokuyo_laser: {sensor_frame: hokuyo_base_link, data_type: LaserScan, clearing: true, marking: true, topic: hokuyo_base/scan, inf_is_valid: true}

inflation:
    inflation_radius: 1.0


Here you have an example of the **costmap_global_static.yaml** file:

In [ ]:
global_frame: map
rolling_window: false
track_unknown_space: true

plugins:
  - {name: static,                  type: "costmap_2d::StaticLayer"}
  - {name: inflation,               type: "costmap_2d::InflationLayer"}

Here you have an example of the **costmap_local.yaml** file:

In [ ]:
global_frame: odom
rolling_window: true

plugins:
  - {name: obstacles_laser,           type: "costmap_2d::ObstacleLayer"}
  - {name: inflation,                 type: "costmap_2d::InflationLayer"}

Here you have an example of **costmap_global_laser.yaml** file:

In [ ]:
global_frame: odom
rolling_window: true
track_unknown_space: true

plugins:
  - {name: obstacles_laser,           type: "costmap_2d::ObstacleLayer"}
  - {name: inflation,                 type: "costmap_2d::InflationLayer"}

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">6. Launch the node and check everything works fine.</p>
</th>
</tr>
</table>

Once you think your parameter files are correct, launch the node and test that the path planning works fine. Check that the robot can plan trajectories and navigate to different points in the environment.

<img src="img/summit_planning1.png" width="800"></img>

<img src="img/summit_planning2.png" width="800"></img>

<img src="img/summit_planning3.png" width="800"></img>

### Step 4: Create a ROS program that interacts with the Navigation Stack

In the Step 2 of this project you created a table with 3 different spots, each one associated with a label. You may have been wondering why you were asked to do that, right? Well, then now you'll get the answer!
<br><br>
In this final Step you will have to create a ROS node that will interact with the Navigation Stack. This node will contain a Service that will take a string as input. This string will indicate one of the labels you selected in Step 2 for the different spots in the simulation. When this service receives a call with the label, it will get the coordinates (position and orientation) associated to that label, and will send this coordinates to the move_base Action Server. If it finished correctly, the service will return an "OK" message.

Summarizing, you will have to create a node which contains:

* A Service that will take an String as input. Those strings will be the labels you selected in Step 2 for each one of the spots.
* An Action Client that will send goals to the move_base Action Server.

In order to achieve this, follow the next steps:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">1. Create the package</p>
</th>
</tr>
</table>

Create the package called <i>my_summit_navigation</i> that will contain all the files related to this project. Remember to specify the <b>rospy</b> dependency.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the service message</p>
</th>
</tr>
</table>

First of all, you'll have to determine the kind of data you need for your mesage.

1. Determine what input data you need ( <span class="ign_blue"><b>request</b></span> )
2. Determine what data you want the service to return ( <span class="ign_blue"><b>response</b></span> )

Next, search if there exists an already built message in the system that suits your needs. If it doesn't, then you'll have to create your own custom message with the data you want.

If this is the case, then do the following. Inside the pacakge you just created, create a new directory named <b>srv</b>. Inside this directory, create a file named <b>MyServiceMessage.srv</b>, that will contain the definition of your service message. 

This file could be something like this:

Now you'll have to modify the <b>package.xml</b> and <b>CMakeLists.txt</b> files of your package, in order to compile the new message. Assuming the only dependency you added to your package is <b>rospy</b>, you should do the following modifications to these files:

#### In CMakeLists.txt

You will have to edit/uncomment four functions inside this file:

* find_package()
* add_service_files()
* generate_messages()
* catkin_package()

add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

generate_messages(
  DEPENDENCIES
  std_msgs
)

#### In package.xml:

Add this 2 lines to the file:

When you've finshed with the modifications, compile your package and check that your message has been correctly compiled and is ready to use. In order to check this, you can use the command <b><i>rossrv show MyServiceMessage</i></b>.

catkin_make output:

<img src="img/catkin_make_output.png" width="600"></img>

rossrv show output:

<img src="img/service_msg_output.png" width="600"></img>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the Service.</p>
</th>
</tr>
</table>

Once your message is created, you're ready to use it in your service! So, let's write the code for our service. Inside the src directory of your package, create a file named <b>my_navigation_program.py</b>. Inside this file, write the code necessary for you Service.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">4. Create the Action Client</p>
</th>
</tr>
</table>

Inside the src directory, create a file that will contain the code of an Action Client.

Once you've finished writting your code, it's time to test it! Call your service providing one of the labels, and check that the Summit robot navigates to that spot. For example, if your label is "fetch_room", type in a web shell the following:

## Add WayPoints

So there you have it. Now your summit XL can navigate around one position at a time. But what if you want to generate in an easy way a sequence of waypoints so that it patroles around the **main interest points of the area**?

You are going to use a ROS package called **follow_waypoints**, you can get more information http://wiki.ros.org/follow_waypoints.
This package basically tracks the **Estimate pose** that you place in RVIZ and stores it. Then when you have finished you just have to publish in a topic that starts sending those positions to the movebase system.

### Steps to do it:

The first step is to start the waypoint server:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1 to move Haro</p><br>
roslaunch follow_waypoints follow_waypoints.launch
</th>
</tr>
</table>

Then you have to start RVIZ, and add a PoseArray element, susbcribed to the topic /waypoints, and change its name to WayPoints. This will now show all the waypoints you set from now on.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2 to move Haro</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

<img src="img/summit_xl_instruction_manual_unit1_waypoints1.png" width="600"></img>

You can change the colour also if you want. In this example its changed to black to see it better.

Now you have to set the waypoints, for this you have to select the **PoseEstimate** and set it where you want.

<img src="img/summit_xl_instruction_manual_unit1_waypoints3.png" width="100"></img>

You should get something similar to this:

<img src="img/summit_xl_instruction_manual_unit1_waypoints2.png" width="600"></img>

You should also get in webshell where you launched the waypoint server a message stating that it recieved the waypoint:

In [ ]:
[INFO][139816504842048][/follow_waypoints/execute:92]: Waiting to recieve waypoints via Pose msg on topic /initialpose         
[INFO][139816504842048][/follow_waypoints/execute:93]: To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -
1'                                                                                                                             
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint

Now you just have to publish in the topic **/path_ready** to start sending waypoints to the movebase.

If you want to see the global path generated just activate it in RVIZ.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3 to move Haro</p><br>
rostopic pub /path_ready std_msgs/Empty -1
</th>
</tr>
</table>

You should get something similar to this for each waypoint executed:

<img src="img/summit_xl_instruction_manual_unit1_waypoints4.png" width="600"></img>

<img src="img/summit_xl_instruction_manual_unit1_waypoints5.png" width="600"></img>

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

Now create a python script that generates those waypoints and when finished executes the moving system to follow those waypoints.<br>
To do so, you will need to publish directly into the topic **/initialpose** where the sserver retreives the data for the waypoint data. Then you will have to publish to start in the topic **/path_ready**.<br>
Make it generate circular paths, eight shaped and dynamic wayponts based on the position of the standing person. For this lastone you will need to acces the TF data of the standingPerson.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

### Extra: Launch your RVIZ configs easily

Here you have an example of how to launch RVIZ sessions faster. You just have to create this tinny launch that retrieves the name of the **rviz** configuration file saved in the **rviz** folder. You will use it in the next units for sure.

**view_robot.launch**

<launch>
  <param name="use_gui" value="true"/>
  <arg name="config" default="complete" />
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find my_summitxl_tools)/rviz/$(arg config).rviz" />
</launch>

And you can use it like so in any launch file:

In [ ]:
<!-- Start RVIZ for Localization -->
<include file="$(find my_summitxl_tools)/launch/view_robot.launch">
    <arg name="config" value="localization" />
</include>

This one will load in RVIZ the file called **localization.rviz**.

### Congratulations! You now have a fully functional indoor navigating Summit XL. In next Chapter you will learn how to navigate outdoors using GPS data.